In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [2]:
ipl2020=pd.read_csv('/content/IPL Ball-by-Ball 2008-2020.csv')
ipl2021=pd.read_csv('/content/IPL Ball-by-Ball Dataset 2021.csv')

In [4]:
matchdata = pd.concat(
    map(pd.read_csv, ['/content/IPL Ball-by-Ball 2008-2020.csv', '/content/IPL Ball-by-Ball Dataset 2021.csv']))

In [ ]:
matchdata.info()

In [7]:
batsmen_score = pd.DataFrame(matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum()).\
            rename(columns={"batsman_runs": "total_runs"})
batsmen_scores_30 = pd.DataFrame(((matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum())>=30).astype(int)).\
    rename(columns={"batsman_runs": "total_runs_30"})
batsmen_scores_50 = pd.DataFrame(((matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum())>=50).astype(int)).\
    rename(columns={"batsman_runs": "total_runs_50"})
batsmen_scores_100 = pd.DataFrame(((matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum())>=100).astype(int)).\
    rename(columns={"batsman_runs": "total_runs_100"})
batsmen_scores_0 = pd.DataFrame(((matchdata.groupby(['id', 'batsman'])['batsman_runs'].sum())==0).astype(int)).\
    rename(columns={"batsman_runs": "total_runs_0"})
batsmen_ball_faced = pd.DataFrame(matchdata.groupby(['id', 'batsman'])['over'].count()).\
    rename(columns={"over": "total_balls_faced"})
batsmen_ball_faced_legal = pd.DataFrame(matchdata[matchdata['ball'] <=6].groupby(['id', 'batsman'])['ball'].count()).\
    rename(columns={"ball": "total_legal_balls_faced"})
batsmen_scores6 = pd.DataFrame(matchdata[matchdata['batsman_runs'] == 6].groupby(['id', 'batsman'])['batsman_runs'].count()).\
    rename(columns={"batsman_runs": "run_6"})
batsmen_scores4 = pd.DataFrame(matchdata[matchdata['batsman_runs'] == 4].groupby(['id', 'batsman'])['batsman_runs'].count()).\
    rename(columns={"batsman_runs": "run_4"})
batsmen_position = pd.DataFrame(matchdata.groupby(['id', 'batsman'])['is_wicket'].min())


df_list = [batsmen_score, batsmen_ball_faced, batsmen_ball_faced_legal, batsmen_scores6, batsmen_scores4, batsmen_position,batsmen_scores_30,batsmen_scores_50,batsmen_scores_100,batsmen_scores_0]
batsmen_summary = pd.concat(df_list, join='outer', axis=1).fillna(np.nan).reset_index()
batsmen_summary = pd.merge(batsmen_summary, matchdata[['id', 'batsman', 'inning', 'batting_team', 'bowling_team']].
                            drop_duplicates(), on=['id', 'batsman'], how='left')
batsmen_summary.rename(columns={'inning': 'batsmen_innings', 'batsman': 'playername', 'batting_team': 'Playing Team',
                                'bowling_team': 'Opponent Team'}, inplace=True)
       

In [43]:
# 0, 0, 1
# Match. Batsman      is_wicket
# 123.    AA Noffke       0
# 123.    AA Noffke       0
# 123.    AA Noffke       1

pd.DataFrame(matchdata.groupby(['id', 'batsman'])['is_wicket'].max())

is_wicket
id      batsman                
335982  AA Noffke             1
        B Akhil               1
        BB McCullum           0
        CL White              1
        DJ Hussey             1
...                         ...
1254078 N Rana                1
        RA Tripathi           1
        Ravi Bishnoi          1
        SP Narine             1
        Shubman Gill          1

[12491 rows x 1 columns]

In [9]:
bowler_wickets = pd.DataFrame(matchdata[((matchdata['is_wicket'] == 1) & (~matchdata['dismissal_kind'].isin(['run out','retired hurt'])))].
                                      groupby(['id', 'bowler'])['is_wicket'].count()).\
    rename(columns={"is_wicket": "total_wickets"})
bowler_overs_bow = pd.DataFrame(matchdata.groupby(['id', 'bowler'])['over'].count()).\
    rename(columns={"over": "total_balls_bowled"})
bowler_ball_faced_legal = pd.DataFrame(matchdata[matchdata['ball'] <=6].groupby(['id', 'bowler'])['ball'].count()).\
    rename(columns={"ball": "total_legal_balls_bowled"})
# calculating the number of maiden overs bowled in a match
bowler_runs_given = pd.DataFrame(matchdata.groupby(['id', 'bowler'])['batsman_runs'].sum()).\
    rename(columns={"batsman_runs": "total_runs_given"})
#matchdata_runs_per_over = pd.DataFrame(self.matchdata.groupby(['matchid', 'bowlername', 'over_num'])['batsmanscorevalue'].sum()).reset_index()
#matchdata_runs_per_over['maiden_overs'] = np.where(matchdata_runs_per_over['batsmanscorevalue'] == 0, 1, 0)
#matchdata_runs_per_over = matchdata_runs_per_over.groupby(['matchid', 'bowlername'])['maiden_overs'].sum()
# combining all the features created so far
df_list = [bowler_wickets, bowler_overs_bow, bowler_ball_faced_legal, bowler_runs_given]
bowler_summary = pd.concat(df_list, join='outer', axis=1).fillna(np.nan).reset_index()
# adding additional columns just in case we need for modeling
bowler_summary = pd.merge(bowler_summary, matchdata[['id', 'bowler', 'inning', 'batting_team', 'bowling_team']].drop_duplicates(), on=['id', 'bowler'], how='left')
bowler_summary.rename(columns={'inning': 'bowlers_innings', 'bowler': 'playername', 'batting_team': 'Playing team',
                                'bowling_team': 'Opponent Team'}, inplace=True)
bowler_summary['economy_rate'] = (bowler_summary['total_runs_given'] * 6) / bowler_summary['total_legal_balls_bowled']

In [58]:
ipl_merged_scorecard = pd.merge(batsmen_summary, bowler_summary, on=['id', 'playername'], how='outer')

In [12]:
MINAVGBALLSFACED = 8
MINAVGBOWLSBOWLED = 6
player_avg = ipl_merged_scorecard[['playername', 'total_balls_faced', 'total_balls_bowled']].fillna(0)
player_avg = pd.DataFrame(player_avg.groupby('playername')[['total_balls_faced', 'total_balls_bowled']].mean())
conditions = [((player_avg['total_balls_faced'] >= MINAVGBALLSFACED) & (player_avg['total_balls_bowled'] >= MINAVGBOWLSBOWLED)),
              (player_avg['total_balls_bowled'] >= MINAVGBOWLSBOWLED)]
choices = ['AllRounder', 'Bowler']
player_avg['playing_role'] = np.select(conditions, choices, default='Batsmen')
player_avg = player_avg.reset_index()

In [14]:
fielder_data = pd.DataFrame(matchdata[((matchdata['is_wicket'] == 1) & (matchdata['dismissal_kind'].isin(['caught','caught and bowled','run out','stumped'])))].
                                      groupby(['fielder','dismissal_kind'])['is_wicket'].count()).\
            rename(columns={"is_wicket": "Fielding_stats"})

In [53]:
# Checking for null values
ipl_merged_scorecard.isnull().sum()

id                              0
playername                      0
total_runs                   5106
total_balls_faced            5106
total_legal_balls_faced      5157
run_6                       12939
run_4                        9949
is_wicket                    5106
total_runs_30                5106
total_runs_50                5106
total_runs_100               5106
total_runs_0                 5106
batsmen_innings              5106
Playing Team                 5106
Opponent Team_x              5119
total_wickets               11995
total_balls_bowled           7678
total_legal_balls_bowled     7678
total_runs_given             7678
bowlers_innings              7678
Playing team                 7678
Opponent Team_y              7690
economy_rate                 7678
dtype: int64

In [ ]:
#id                              0
#playername                      0

# Null -> 0
#----------------------------------
#total_runs                   5106
#total_balls_faced            5106
#total_legal_balls_faced      5157
#run_6                       12939
#run_4                        9949
#total_runs_30                5106
#total_runs_50                5106
#total_runs_100               5106
#total_runs_0                 5106
#total_wickets               11995
#total_balls_bowled           7678
#total_legal_balls_bowled     7678
#total_runs_given             7678
#economy_rate                 7678

#----------------------------------

# Batsman_summary
# Not in Bowler_summary - join -> Null - 0?
#is_wicket                    5106
#batsmen_innings              5106

#----------------------------------
# Bowler_summary
# Not in Batsman_summary - join -> Null - 0?
#bowlers_innings              7678

#----------------------------------
# Swap for batmen who didnt bowl OR bowlers who didnt bat?
#Playing Team                 5106
#Opponent Team_x              5119
#Playing team                 7678
#Opponent Team_y              7690


In [51]:
# List of cols where null -> 0
list_of_cols = ipl_merged_scorecard.columns[2:]
list_of_cols = list(list_of_cols)
#list_of_cols.remove("")
list_of_cols

['total_runs',
 'total_balls_faced',
 'total_legal_balls_faced',
 'run_6',
 'run_4',
 'is_wicket',
 'total_runs_30',
 'total_runs_50',
 'total_runs_100',
 'total_runs_0',
 'batsmen_innings',
 'Playing Team',
 'Opponent Team_x',
 'total_wickets',
 'total_balls_bowled',
 'total_legal_balls_bowled',
 'total_runs_given',
 'bowlers_innings',
 'Playing team',
 'Opponent Team_y',
 'economy_rate']

In [55]:
# in case all cols Nulls -> 0
for col in list_of_cols:
  #ipl_merged_scorecard.loc[ ipl_merged_scorecard[col].isnull() , col ] = 0
  ipl_merged_scorecard[col].fillna(0, inplace=True)
ipl_merged_scorecard.isnull().sum()

id                          0
playername                  0
total_runs                  0
total_balls_faced           0
total_legal_balls_faced     0
run_6                       0
run_4                       0
is_wicket                   0
total_runs_30               0
total_runs_50               0
total_runs_100              0
total_runs_0                0
batsmen_innings             0
Playing Team                0
Opponent Team_x             0
total_wickets               0
total_balls_bowled          0
total_legal_balls_bowled    0
total_runs_given            0
bowlers_innings             0
Playing team                0
Opponent Team_y             0
economy_rate                0
dtype: int64

In [59]:
# Swap for batmen who didnt bowl OR bowlers who didnt bat?
#Playing Team                 5106
#Opponent Team_x              5119
#Playing team                 7678
#Opponent Team_y              7690
team_cols = ["Playing Team", "Opponent Team_x", "Playing team", "Opponent Team_y" ]
ipl_merged_scorecard[["Playing Team", "Opponent Team_x"]].isnull().sum()


Playing Team       5106
Opponent Team_x    5119
dtype: int64

In [60]:
ipl_merged_scorecard[["Playing team", "Opponent Team_y"]].isnull().sum()

Playing team       7678
Opponent Team_y    7690
dtype: int64

In [62]:
ipl_merged_scorecard [ (ipl_merged_scorecard["Playing Team" ].isna() == False) & (ipl_merged_scorecard["Opponent Team_x"].isna()) ]

,id,playername,total_runs,total_balls_faced,total_legal_balls_faced,run_6,run_4,is_wicket,total_runs_30,total_runs_50,...,Playing Team,Opponent Team_x,total_wickets,total_balls_bowled,total_legal_balls_bowled,total_runs_given,bowlers_innings,Playing team,Opponent Team_y,economy_rate
3635,501265,CA Ingram,13.0,12.0,12.0,NaN,1.0,0.0,0.0,0.0,...,Delhi Daredevils,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3636,501265,DA Warner,19.0,10.0,10.0,2.0,1.0,0.0,0.0,0.0,...,Delhi Daredevils,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3637,501265,MS Wade,11.0,20.0,19.0,NaN,NaN,0.0,0.0,0.0,...,Delhi Daredevils,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3638,501265,NV Ojha,8.0,20.0,20.0,NaN,1.0,0.0,0.0,0.0,...,Delhi Daredevils,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3639,501265,Y Venugopal Rao,0.0,1.0,NaN,NaN,NaN,0.0,0.0,0.0,...,Delhi Daredevils,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7296,829763,AB de Villiers,57.0,47.0,45.0,1.0,9.0,0.0,1.0,1.0,...,Royal Challengers Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7297,829763,CH Gayle,10.0,4.0,4.0,1.0,1.0,0.0,0.0,0.0,...,Royal Challengers Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7298,829763,D Wiese,11.0,9.0,9.0,NaN,2.0,0.0,0.0,0.0,...,Royal Challengers Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7299,829763,HV Patel,6.0,2.0,2.0,1.0,NaN,0.0,0.0,0.0,...,Royal Challengers Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7300,829763,KD Karthik,27.0,19.0,17.0,1.0,3.0,0.0,0.0,0.0,...,Royal Challengers Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
ipl_merged_scorecard[ ipl_merged_scorecard["id"] == 501265]

,id,playername,total_runs,total_balls_faced,total_legal_balls_faced,run_6,run_4,is_wicket,total_runs_30,total_runs_50,...,Playing Team,Opponent Team_x,total_wickets,total_balls_bowled,total_legal_balls_bowled,total_runs_given,bowlers_innings,Playing team,Opponent Team_y,economy_rate
3635,501265,CA Ingram,13.0,12.0,12.0,NaN,1.0,0.0,0.0,0.0,...,Delhi Daredevils,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3636,501265,DA Warner,19.0,10.0,10.0,2.0,1.0,0.0,0.0,0.0,...,Delhi Daredevils,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3637,501265,MS Wade,11.0,20.0,19.0,NaN,NaN,0.0,0.0,0.0,...,Delhi Daredevils,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3638,501265,NV Ojha,8.0,20.0,20.0,NaN,1.0,0.0,0.0,0.0,...,Delhi Daredevils,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3639,501265,Y Venugopal Rao,0.0,1.0,NaN,NaN,NaN,0.0,0.0,0.0,...,Delhi Daredevils,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13867,501265,AC Thomas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,18.0,18.0,15.0,1.0,Delhi Daredevils,NaN,5.000000
13868,501265,JP Faulkner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,12.0,12.0,9.0,1.0,Delhi Daredevils,NaN,4.500000
13869,501265,MR Marsh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,13.0,13.0,11.0,1.0,Delhi Daredevils,NaN,5.076923
13870,501265,R Sharma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,6.0,6.0,4.0,1.0,Delhi Daredevils,NaN,4.000000
13871,501265,SS Mundhe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,7.0,6.0,5.0,1.0,Delhi Daredevils,NaN,5.000000


In [63]:
ipl_merged_scorecard[ ipl_merged_scorecard["id"] == 829763]

,id,playername,total_runs,total_balls_faced,total_legal_balls_faced,run_6,run_4,is_wicket,total_runs_30,total_runs_50,...,Playing Team,Opponent Team_x,total_wickets,total_balls_bowled,total_legal_balls_bowled,total_runs_given,bowlers_innings,Playing team,Opponent Team_y,economy_rate
7296,829763,AB de Villiers,57.0,47.0,45.0,1.0,9.0,0.0,1.0,1.0,...,Royal Challengers Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7297,829763,CH Gayle,10.0,4.0,4.0,1.0,1.0,0.0,0.0,0.0,...,Royal Challengers Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7298,829763,D Wiese,11.0,9.0,9.0,NaN,2.0,0.0,0.0,0.0,...,Royal Challengers Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7299,829763,HV Patel,6.0,2.0,2.0,1.0,NaN,0.0,0.0,0.0,...,Royal Challengers Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7300,829763,KD Karthik,27.0,19.0,17.0,1.0,3.0,0.0,0.0,0.0,...,Royal Challengers Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7301,829763,Mandeep Singh,27.0,21.0,20.0,NaN,4.0,0.0,0.0,0.0,...,Royal Challengers Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7302,829763,SN Khan,45.0,22.0,19.0,1.0,6.0,0.0,1.0,0.0,...,Royal Challengers Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7303,829763,V Kohli,1.0,4.0,4.0,NaN,NaN,0.0,0.0,0.0,...,Royal Challengers Bangalore,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15358,829763,DS Kulkarni,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,26.0,24.0,34.0,1.0,Royal Challengers Bangalore,NaN,8.500000
15359,829763,JP Faulkner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,13.0,12.0,25.0,1.0,Royal Challengers Bangalore,NaN,12.500000
